![](./img/dl_banner.jpg)

# Keras卷积神经网络
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

## 0.知识背景：卷积神经网络
卷积神经网络是一种在计算机视觉当中广泛应用的神经网络，其特殊的网络结构，包含卷积层、池化层等，能在共享参数的同时保证对图像特征的高效抽取。经典的卷积神经网络结构如下。
![CNN](http://personal.ie.cuhk.edu.hk/~ccloy/project_target_code/images/fig3.png)

## 0.数据背景：Cifar-10
cifar-10是一个很著名的实验数据集，该数据集共有60000张彩色图像，这些图像是32\*32，分为10个类，每类6000张图。这里面有50000张用于训练，构成了5个训练批，每一批10000张图；另外10000用于测试，单独构成一批。测试批的数据里，取自10类中的每一类，每一类随机取1000张。抽剩下的就随机排列组成了训练批。注意一个训练批中的各类图像并不一定数量相同，总的来看训练批，每一类都有5000张图。
![](./img/cifar-10.png)

## 1.加载数据
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

In [2]:
from keras.datasets import cifar10
from keras.utils import np_utils
from keras import Sequential
from keras.layers import Conv2D, Activation, Flatten, Dense, Dropout, MaxPool2D
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

Using TensorFlow backend.


注意：直接通过“(X_train, y_train), (X_test, y_test) = cifar10.load_data()”加载数据需要科学上网。

这里，可以进行如下操作加载数据：

1. 运行cell 2
2. 3-5秒后停止运行（点击上方工具栏中的黑色正方形按钮）
3. 运行cell 3，并等待运行完成
4. 重新运行cell 2

完成上述操作后，即加载数据成功。

In [12]:
# cell 2
# 加载数据
nb_classes = 10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
# 幅度缩放
X_train /= 255
X_test /= 255

In [11]:
# cell 3
! cp /data/Deep_learning/cifar-10-batches-py.tar.gz  ~/.keras/datasets/cifar-10-batches-py.tar.gz


## 2.数据增强
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

为了提高模型的泛化能力，我们使用到了一个叫做`数据增强`的处理，在keras中可以通过ImageDataGenerator完成，详细的可以参见[Image Data Generator](http://keras.io/preprocessing/image/)。这个处理会对输入的图片数据进行例如旋转、平移、截取、水平、垂直翻转等处理，并不改变图像内容，但是能扩量计算机没有看过的数据，提高模型的泛化能力。

#### 提示：这个部分可能计算量比较大....

In [8]:
from keras.preprocessing.image import ImageDataGenerator

generated_images = ImageDataGenerator(
    featurewise_center=True,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=True,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

generated_images.fit(X_train)

我们每次从`ImageDataGenerator`中取出500张做过数据增强的图片用于一个批次的训练。

In [9]:
X_train.shape

(50000, 32, 32, 3)

In [10]:
gen = generated_images.flow(X_train, Y_train, batch_size=500, shuffle=True)
X_batch, Y_batch = next(gen)

In [11]:
X_batch.shape

(500, 32, 32, 3)

## 3.构建模型
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

In [12]:
nb_epoch = 10  # 迭代轮次
batch_size = 128 # 一个batch的数据量
nb_filters = 32 # 卷积层filter的个数
nb_pool = 2 # 池化层的kernel size
nb_conv = 3

In [13]:
model = Sequential()
model.add(Conv2D(nb_filters, (nb_conv, nb_conv),padding='same',input_shape=X_batch.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPool2D(2,2))
model.add(Conv2D(nb_filters, (nb_conv, nb_conv)))
model.add(Activation('relu'))
model.add(MaxPool2D(2,2))
model.add(Conv2D(nb_filters, (nb_conv, nb_conv)))
model.add(Activation('relu'))
model.add(MaxPool2D(2,2))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

## 4.模型训练与评估
#### \[稀牛学院 x 网易云课程\]《深度学习工程师(实战)》课程资料 by [@寒小阳](https://blog.csdn.net/han_xiaoyang)

In [17]:
model.fit_generator(gen, epochs=20, steps_per_epoch=100)

Epoch 1/20
100/100 [==============================] - 28s 280ms/step - loss: 1.0306 - acc: 0.6453
Epoch 2/20
100/100 [==============================] - 28s 281ms/step - loss: 1.0193 - acc: 0.6443
Epoch 3/20
100/100 [==============================] - 28s 280ms/step - loss: 1.0274 - acc: 0.6427
Epoch 4/20
100/100 [==============================] - 28s 280ms/step - loss: 1.0101 - acc: 0.6502
Epoch 5/20
100/100 [==============================] - 28s 283ms/step - loss: 1.0137 - acc: 0.6487
Epoch 6/20
100/100 [==============================] - 28s 284ms/step - loss: 1.0140 - acc: 0.6468
Epoch 7/20
100/100 [==============================] - 28s 283ms/step - loss: 1.0115 - acc: 0.6494
Epoch 8/20
100/100 [==============================] - 28s 283ms/step - loss: 0.9941 - acc: 0.6544
Epoch 9/20
100/100 [==============================] - 28s 282ms/step - loss: 0.9983 - acc: 0.6507
Epoch 10/20
100/100 [==============================] - 29s 285ms/step - loss: 0.9959 - acc: 0.6571
Epoch 11/20
100/100

In [20]:
model.evaluate(X_train, Y_train)

50000/50000 [==============================] - 4s 82us/step


[2.211750121612549, 0.30754]

In [21]:
model.evaluate(X_test, Y_test)

10000/10000 [==============================] - 1s 84us/step


[2.2134314895629883, 0.3077]

### 版权归 © 稀牛学院 所有 保留所有权利
![](./img/xiniu_neteasy.png)